In [0]:
from pyspark.sql.functions import col,lit,max

In [0]:
df = spark.sql("""
               select * from gizmobox.bronze.py_customers
               """)

In [0]:
df = spark.read.table("gizmobox.bronze.py_customers")

In [0]:
df_filtered = df.where("customer_id is not null") \
                .dropDuplicates()

In [0]:
df_groupby = df_filtered.groupBy(col("customer_id")) \
                        .agg(max("created_timestamp").alias("created_timestamp"))
df_final = df_groupby.alias("d1").join(df_filtered.alias("d2")) \
                     .where((col("d1.customer_id") == col("d2.customer_id")) & (col("d1.created_timestamp") == col("d2.created_timestamp"))) \
                     .select(col("d2.created_timestamp").cast("timestamp"),col("d2.customer_id"),col("d2.customer_name"),col("d2.email"),col("d2.date_of_birth").cast("date"),col("d2.member_since").cast("date"),col("d2.telephone"))


   

In [0]:
df_groupby = df_filtered.groupBy(col("customer_id")) \
                        .agg(max("created_timestamp").alias("created_timestamp"))
df_final = df_groupby.alias("d1").join(df_filtered.alias("d2"),(col("d1.customer_id") == col("d2.customer_id")) &   (col("d1.created_timestamp") == col("d2.created_timestamp"))) \
                     .select(col("d2.created_timestamp").cast("timestamp"),col("d2.customer_id"),col("d2.customer_name"),col("d2.email"),col("d2.date_of_birth").cast("date"),col("d2.member_since").cast("date"),col("d2.telephone"))


In [0]:
df_final.writeTo("gizmobox.silver.py_customers").createOrReplace()

In [0]:
df_cust = spark.table("gizmobox.silver.py_customers")

In [0]:
%sql
select * from gizmobox.silver.py_customers